In [ ]:
# Extract zip files
import zipfile
with zipfile.ZipFile('./acousticbrainz.zip', 'r') as zip:
    zip.extractall('extracted')

In [6]:
# Extract subfiles and move everything around to be more convenient
import tarfile
from os import listdir, mkdir, replace
import bz2

mkdir('./tsvs')
for filename in listdir('./extracted'):
    if '.tar.bz2' in filename:
      tarfile.open('./extracted/' + filename).extractall('./train' if 'train' in filename else './validation')
    elif 'tsv.bz2' in filename:
      data = bz2.BZ2File('./extracted/' + filename).read()
      open('./tsvs/' + filename[:-4], 'wb').write(data)
    
for folder in listdir('./train/acousticbrainz-mediaeval-train'):
    for filename in listdir('./train/acousticbrainz-mediaeval-train/' + folder):
        replace('./train/acousticbrainz-mediaeval-train/' + folder + '/' + filename, './train/' + filename)

for folder in listdir('./validation/acousticbrainz-mediaeval-validation'):
    for filename in listdir('./validation/acousticbrainz-mediaeval-validation/' + folder):
        replace('./validation/acousticbrainz-mediaeval-validation/' + folder + '/' + filename, './validation/' + filename)

In [189]:
# Go through the data and preprocess it into a usable form
import json
from os import listdir
import numpy as np

properties = [
    'metadata/audio_properties/length',
    'rhythm/bpm',
    'rhythm/danceability',
    'tonal/key_scale', # 0=minor or 1=major
    # Key_key, key_scale, chords_key, chords_scale are all
]
"""
    'rhythm/onset_rate',
    'tonal/tuning_equal_tempered_deviation',
    'tonal/chords_changes_rate',
    'tonal/tuning_diatonic_strength',
    'tonal/chords_number_rate',
    'tonal/tuning_frequency',
    'tonal/tuning_nontempered_energy_ratio',
    'tonal/key_strength',
    'tonal/chords_strength/dmean2',
    'tonal/chords_strength/median',
    'tonal/chords_strength/min',
    'tonal/chords_strength/dvar2',
    'tonal/chords_strength/dvar',
    'tonal/chords_strength/dmean',
    'tonal/chords_strength/max',
    'tonal/chords_strength/var',
    'tonal/chords_strength/mean',
    'tonal/hpcp_entropy/dmean2',
    'tonal/hpcp_entropy/median',
    'tonal/hpcp_entropy/min',
    'tonal/hpcp_entropy/dvar2',
    'tonal/hpcp_entropy/dvar',
    'tonal/hpcp_entropy/dmean',
    'tonal/hpcp_entropy/max',
    'tonal/hpcp_entropy/var',
    'tonal/hpcp_entropy/mean',
"""

X_train = []
X_validation = []

y_train = []
y_validation = []

all_genres = set()

def process_tsv(file_path, is_validation=False):
    with open(file_path) as tsv:
        tsv.readline()
        for _ in range(100000):
            line = tsv.readline()
            if not line:
                break
            entries = line.split('\t')
            
            genres = [genre for genre in entries[2:] if genre != '' and genre != '\n']
            for genre in genres:
                all_genres.add(genre)
            if not is_validation:
                y_train.append(genres)
            else:
                y_validation.append(genres)


            with open(('./train/' if not is_validation else './validation/') + entries[0] + '.json') as file:
                song = json.load(file)
                datapoint = []
                for property in properties:
                    path = property.split('/')

                    temp = song
                    for partial in path:
                        temp = temp[partial]

                    if partial == 'key_scale': # edge case
                        if temp == 'minor':
                            temp = 0
                        else:
                            temp = 1

                    datapoint.append(temp)
                datapoint = np.array(datapoint)
                if is_validation:
                    X_validation.append(datapoint)
                else:
                    X_train.append(datapoint)

process_tsv('./tsvs/acousticbrainz-mediaeval-discogs-train.tsv')
process_tsv('./tsvs/acousticbrainz-mediaeval-discogs-validation.tsv', is_validation=True)

all_genres = list(all_genres) # contains all the genres and their indices
for i, datapoint in enumerate(y_train):
    processed_datapoint = []
    for genre in all_genres:
        processed_datapoint.append(int(genre in datapoint))
    y_train[i] = processed_datapoint

for i, datapoint in enumerate(y_validation):
    processed_datapoint = []
    for genre in all_genres:
        processed_datapoint.append(int(genre in datapoint))
    y_validation[i] = processed_datapoint

X_train = np.array(X_train)
X_validation = np.array(X_validation)

y_train = np.array(y_train)
y_validation = np.array(y_validation)

In [194]:
# Define the model and give a summary
from keras import layers
from keras.models import Model

input = layers.Input(shape=(len(properties),))
model = Model(input, layers.Dense(len(all_genres), activation='relu')(input))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "model_45"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_74 (InputLayer)       [(None, 30)]              0         
                                                                 
 dense_45 (Dense)            (None, 318)               9858      
                                                                 
Total params: 9,858
Trainable params: 9,858
Non-trainable params: 0
_________________________________________________________________


In [195]:
# Train and save the model 
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_validation, y_validation))
score = model.evaluate(X_validation, y_validation, verbose=1)
print('Validation loss and accuracy:', score)
model.save('trained_model')

Epoch 1/10
3125/3125 [==============================] - 5s 1ms/step - loss: 3.2764 - accuracy: 0.0000e+00 - val_loss: 3.0804 - val_accuracy: 0.0000e+00
Epoch 2/10
3125/3125 [==============================] - 4s 1ms/step - loss: 3.0797 - accuracy: 0.0000e+00 - val_loss: 3.0802 - val_accuracy: 0.0000e+00
Epoch 3/10
3125/3125 [==============================] - 4s 1ms/step - loss: 3.0796 - accuracy: 0.0000e+00 - val_loss: 3.0802 - val_accuracy: 0.0000e+00
Epoch 4/10
3125/3125 [==============================] - 4s 1ms/step - loss: 3.0796 - accuracy: 0.0000e+00 - val_loss: 3.0802 - val_accuracy: 0.0000e+00
Epoch 5/10
3125/3125 [==============================] - 4s 1ms/step - loss: 3.0796 - accuracy: 0.0000e+00 - val_loss: 3.0802 - val_accuracy: 0.0000e+00
Epoch 6/10
3125/3125 [==============================] - 4s 1ms/step - loss: 3.0796 - accuracy: 0.0000e+00 - val_loss: 3.0802 - val_accuracy: 0.0000e+00
Epoch 7/10
3125/3125 [==============================] - 4s 1ms/step - loss: 3.0796 - acc

In [184]:
print(y_train[0])
result = model.predict(np.array(X_train[0]).reshape(1,len(properties)))
print(len(result[0]))
print(result)
print(max(result[0]))
print(list(result[0]).index(max(result[0])))
print(all_genres[17])
print(y_train[0][17])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
318
[[74.340355   0.         0.         7.9846654 30.34112    0.
   0.         0.         0.         0.         0.         0.
   0.         0.        62.462246   0.         0.        75.62224
   0.         0.        48.710922   0.        56.399654   0.
  63.26691   43.401943  51.139717   0.        22.255716   0.
   0.        49.966183   0.         0.  